In [1]:
import pandas as pd
import boto3
import numpy as np

In [2]:
df_eventos=pd.read_csv('eventos_amplitude.csv')

In [3]:
# EVENTOS CON LOS QUE VAMOS A TRABAJAR

bucket_amplitude_data='churn-ds-stage'
key='ar/amplitude/tb_ar_amplitude_events_stage/'

eventos_amplitude={
   'cuotificaciones': [i for i in df_eventos.event_type if i.find('cuoti_sigue_seleccion_consumos')>=0 or i.find('cuoti_selecciona_elegircuo')>=0 or i == 'cuoti_ingresa_clave' ],
   'prestamos' : [i for i in df_eventos.event_type if i == 'prestamos_selecciona_simular_prestamo' or i == 'prestamos_espera' or i == 'prestamos_ingresa_clave'],
   'inversiones' : [i for i in df_eventos.event_type if i.find('click_inversiones')>=0 or i == 'inversiones_suscribir'],
   'promociones' : [i for i in df_eventos.event_type if i.find('general_ingresa_promoc')>=0],
   'recargas' : [i for i in df_eventos.event_type if i.find('recargas_click_empezar')>=0 or i.find('recargas_click_repetir')>=0 or i.find('recarga_confirmada')>=0],
   'transferencias_c2c' : [i for i in df_eventos.event_type if i.find('transferencia_selecciona_tieneuala')>=0 or i.find('transferencia_validar_identidad')>=0],
   'transferencias_cvu' : [i for i in df_eventos.event_type if i.find('transferencia_selecciona_notieneuala')>=0 or i.find('transferencia_validar_identidad')>=0],
   'cobros' : [i for i in df_eventos.event_type if i.find('cobros_acepta_tyc')>=0 or i.find('general_ingresa_cobros')>=0 or i.find('cobros_elige_link')>=0 or i.find('cobros_elige_mpos')>=0],
   'pago_servicios' : [i for i in df_eventos.event_type if i.find('pagos_empezar')>=0 or i.find('pago_confirmado')>=0]
  }

s3=boto3.resource('s3')

In [4]:
eventos_amplitude

{'cuotificaciones': ['cuoti_ingresa_clave',
  'cuoti_selecciona_elegircuotas',
  'cuoti_sigue_seleccion_consumos'],
 'prestamos': ['prestamos_espera',
  'prestamos_selecciona_simular_prestamo',
  'prestamos_ingresa_clave'],
 'inversiones': ['inversiones_suscribir', 'click_inversiones'],
 'promociones': ['general_ingresa_promociones'],
 'recargas': ['recargas_click_empezar',
  'recarga_confirmada',
  'recargas_click_repetir'],
 'transferencias_c2c': ['transferencia_validar_identidad',
  'transferencia_selecciona_tieneuala'],
 'transferencias_cvu': ['transferencia_selecciona_notieneuala',
  'transferencia_validar_identidad'],
 'cobros': ['general_ingresa_cobros',
  'cobros_acepta_tyc',
  'cobros_elige_link',
  'cobros_elige_mpos'],
 'pago_servicios': ['pago_confirmado', 'pagos_empezar']}

In [9]:
bucket=s3.Bucket(bucket_amplitude_data)

In [10]:
# BUSCAR LOS ARCHIVOS DEL BUCKET 
files_bucket=list(bucket.objects.all())

In [11]:
# BUSCAR LOS ARCHIVOS DE AMPLITUDE
files_amplitude=[i.key for i in files_bucket if i.key.find(key)>=0 and i.key.find('.parquet')>=0]

In [12]:
# FECHA DE LOS ARCHIVOS DE AMPLITUDE
file_amplitude_fecha=pd.to_datetime([i[i.find('dt=')+3:i.find('dt=')+13] for i in files_amplitude])

In [13]:
df_file_amplitude=pd.DataFrame({'file':files_amplitude,
                               'fecha':file_amplitude_fecha})
df_file_amplitude.head()

,file,fecha
0,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-05-01
1,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-05-01
2,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-05-01
3,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-05-01
4,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-05-01


In [14]:
#Fecha inicial
df_file_amplitude.fecha.min()

Timestamp('2020-05-01 00:00:00')

In [15]:
#Fecha final
df_file_amplitude.fecha.max()

Timestamp('2021-03-31 00:00:00')

In [16]:
file_ventana_total=df_file_amplitude.loc[(df_file_amplitude.fecha>='2020-05-01') &  
                                      (df_file_amplitude.fecha<='2021-03-31'),:]

In [26]:
falsa_ventana=df_file_amplitude.loc[(df_file_amplitude.fecha>='2020-06-01') &  
                                      (df_file_amplitude.fecha<='2020-06-03'),:]

In [27]:
falsa_ventana

,file,fecha
743,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-01
744,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-01
745,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-01
746,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-01
747,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-01
...,...,...
810,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-03
811,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-03
812,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-03
813,ar/amplitude/tb_ar_amplitude_events_stage/dt=2...,2020-06-03


In [19]:
file_ventana_total[file_ventana_total.fecha=='2021-03-31'].shape

(24, 2)

**Resumen de Datos del mes**

In [20]:
print('cantidad de filas:',file_ventana_total.shape[0],'cantidad de columnas:',file_ventana_total.shape[1])

cantidad de filas: 7905 cantidad de columnas: 2


In [21]:
print('cantidad de dias:',len(file_ventana_total.fecha.unique()))

cantidad de dias: 327


In [22]:
print('Cantidad promedio de archivos por dia:',file_mes_actual.shape[0]/len(file_mes_actual.fecha.unique()))

NameError: name 'file_mes_actual' is not defined

**Consideraciones**

1. Seleccionar los eventos que vamos a tomar.
2. Ver si se pueden agrupar los eventos.
3. Esto no se debe hacer porque no permite tantos procesos en concurrencia.

[p1] [p2] [p3] [p4] .. [p644]
 |    |.    |.   |        |
[t1] [t2] [t3] [t4]....[t644]
 |_  _ _ _ _ _ _ |........|
          |
  [parquet mensual]
  
4. Filtro el dataframe con eventos por dia.

[p1] [p2] [p3] [p4] .. [p24]
 |    |.    |.   |        |
[t1] [t2] [t3] [t4]....[t24]
 |_  _ _ _ _ _ _ |........|
          |
  [parquet diaria 1]   [parquet diaria 2]    [parquet diaria 3]  ---  [parquet diaria 31]
          |  -- - - - - - - - | --- - - - - - -  - - | -- - - - - - - - - -- - |
                                  [parquete mensual]


**Dask**

In [24]:
import dask.dataframe as dd
import dask.array as da
pd.set_option('display.max_columns', None)
import gc

In [28]:
# los dias
dates_file=falsa_ventana.fecha.unique()
d=dates_file[0]

# lista de archivos
l=list(falsa_ventana.loc[falsa_ventana.fecha==d,'file'].values)
l=[f's3://{bucket_amplitude_data}/'+i for i in l]

#columnas
columns_=['user_id','os_name','event_type','device_carrier','event_time']

#Eventos a filtrar
events_total=[]
for i in eventos_amplitude.keys():
        events_total.extend(eventos_amplitude[i])

In [37]:
events_total=list(set(events_total))

In [38]:
len(events_total)

21

In [81]:
# GENERAR EL DASK DATAFRAME
data_amplitude_diaria=dd.read_parquet(l,columns=columns_,engine='pyarrow')

In [82]:
# FILTRAMOS LOS EVENTOS DE AMPLITUDE
data_amplitude_diaria=data_amplitude_diaria.loc[data_amplitude_diaria.event_type.isin(events_total),:]

In [166]:
print('PASO 8: Generar el tamaño de los chunks')
chunks = tuple(data_amplitude_diaria.map_partitions(len).compute())    

PASO 8: Generar el tamaño de los chunks


In [83]:
data_amplitude_diaria.compute().head()

,user_id,os_name,event_type,device_carrier,event_time
1,901aab34-c1e3-4028-af11-546cbbbd598d,android,recarga_confirmada,Claro AR,2020-06-01 20:59:55.893000
13,94bb12bb-62d6-4e87-9284-0a339b15bf71,android,click_inversiones,Movistar,2020-06-01 20:59:56.341000
80,3c4d64ee-2149-4326-bb3b-88b110c3d7d2,android,pagos_empezar,Claro AR,2020-06-01 20:59:59.550000
91,2e27287c-2127-4e51-9905-36f3d2bad686,android,click_inversiones,Tuenti,2020-06-01 20:59:59.995000
96,7192c5ec-02cb-42c2-81ce-d6dd56370cb8,android,recarga_confirmada,<NA>,2020-06-01 21:00:04.273000


In [91]:
data_amplitude_diaria=data_amplitude_diaria.assign(event_cast=data_amplitude_diaria.event_time.str[:13])

In [ ]:
data_group=dd.DataFrameGroupBy(data_amplitude_diaria,by=['user_id','os_name'])

In [100]:
data_amplitude_diaria.groupby(['user_id','event_cast']).agg('count').compute().head()

os_name  \
user_id                              event_type                       
0005571a-5d34-4f01-af08-dca0397b5824 click_inversiones            1   
0005c5f3-43cb-4bb9-bc13-15a5baf67ed7 click_inversiones            6   
00089d76-24a0-4824-99dd-ab3d5fae9839 click_inversiones            1   
                                     inversiones_suscribir        1   
000b4772-e4e0-4249-b632-0cadef1e95a5 click_inversiones            5   

                                                            device_carrier  \
user_id                              event_type                              
0005571a-5d34-4f01-af08-dca0397b5824 click_inversiones                   1   
0005c5f3-43cb-4bb9-bc13-15a5baf67ed7 click_inversiones                   6   
00089d76-24a0-4824-99dd-ab3d5fae9839 click_inversiones                   1   
                                     inversiones_suscribir               1   
000b4772-e4e0-4249-b632-0cadef1e95a5 click_inversiones                   5   

                                                            event_time  \
user_id                              event_type                          
0005571a-5d34-4f01-af08-dca0397b5824 click_inversiones               1   
0005c5f3-43cb-4bb9-bc13-15a5baf67ed7 click_inversiones               6   
00089d76-24a0-4824-99dd-ab3d5fae9839 click_inversiones               1   
                                     inversiones_suscribir           1   
000b4772-e4e0-4249-b632-0cadef1e95a5 click_inversiones               5   

                                                            event_cast  
user_id                              event_type                         
0005571a-5d34-4f01-af08-dca0397b5824 click_inversiones               1  
0005c5f3-43cb-4bb9-bc13-15a5baf67ed7 click_inversiones               6  
00089d76-24a0-4824-99dd-ab3d5fae9839 click_inversiones               1  
                                     inversiones_suscribir           1  
000b4772-e4e0-4249-b632-0cadef1e95a5 click_inversiones               5

In [102]:
data_amplitude_diaria2 = data_amplitude_diaria.categorize("event_type")
data_amplitude_diaria3 = data_amplitude_diaria2.pivot_table(index='user_id', columns="event_type",aggfunc='count')


In [103]:
data_amplitude_diaria3.compute().head()

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [167]:
cond=np.where(data_amplitude_diaria['event_type'].isin(eventos_amplitude['ingreso_clave']),1,0)
cond_dask=da.from_array(cond,chunks=chunks)
data_amplitude_diaria=data_amplitude_diaria.assign(ingreso_clave=cond_dask)

In [169]:
cond=np.where(data_amplitude_diaria['event_type'].isin(eventos_amplitude['ingreso_app']),1,0)
cond_dask=da.from_array(cond,chunks=chunks)
data_amplitude_diaria=data_amplitude_diaria.assign(ingreso_app=cond_dask)

In [75]:
data_amplitude_diaria_2=data_amplitude_diaria.assign(event_cast=data_amplitude_diaria.event_time.str[:13])

In [76]:
data_amplitude_diaria_2.compute().head()

,user_id,os_name,event_type,device_carrier,event_time,event_cast
1,901aab34-c1e3-4028-af11-546cbbbd598d,android,recarga_confirmada,Claro AR,2020-06-01 20:59:55.893000,2020-06-01 20
13,94bb12bb-62d6-4e87-9284-0a339b15bf71,android,click_inversiones,Movistar,2020-06-01 20:59:56.341000,2020-06-01 20
80,3c4d64ee-2149-4326-bb3b-88b110c3d7d2,android,pagos_empezar,Claro AR,2020-06-01 20:59:59.550000,2020-06-01 20
91,2e27287c-2127-4e51-9905-36f3d2bad686,android,click_inversiones,Tuenti,2020-06-01 20:59:59.995000,2020-06-01 20
96,7192c5ec-02cb-42c2-81ce-d6dd56370cb8,android,recarga_confirmada,<NA>,2020-06-01 21:00:04.273000,2020-06-01 21


In [ ]:
rint('PASO 11: Guardar en S3 archivo con datos de amplitude')
dd.to_parquet(base_raw,path=f's3://{survival_bucket}/data/raw/amplitude/',
              compression='snappy',
              write_index=False,
              overwrite=False,
              append=False,
              partition_on=['dt'],
              storage_options={"s3_additional_kwargs":{
                             'ServerSideEncryption': 'aws:kms',
                             'SSEKMSKeyId': kms_key_arn}})
